# Tutorial Intel Advisor

Neste exemplo vamos conhecer a ferramenta Intel Advisor usando um exemplo de multiplicação de matrizes 1024x1024. Execute a célula a seguir uma única vez para criar a pasta `build`:


In [ ]:
!mkdir build

Os próximos passos serão executados repetidas vezes, uma para cada versão do código. Para modificar a versão desejada, [edite este arquivo](src/multiply.hpp) na linha 35.  Os valores possíveis são `multiply1`, `multiply1_1` e `multiply1_2`.

In [ ]:
!cd build && cmake .. && make
!advisor --collect=roofline --project-dir=./advi  --search-dir src:p=./advi -- ./build/matrix.dpcpp
!advisor --report=roofline --report-output=./advi/advisor-roofline.html --project-dir=./advi

In [ ]:
from IPython.display import IFrame; IFrame('advi/advisor-roofline.html', width='100%', height=600)

## Análise

A primeira versão (`multiply1`) está limitada à vazão da memória principal (DRAM). A intensidade operacional é 0.242 FLOP/Byte.

A segunda versão (`multiply1_1`) usa uma variável local como acumulador o que, supostamente, aumenta a intensidade operacional para 0.471 FLOP/Byte, mas não melhora o desempenho. Isso ocorre porque o gargalo são os acessos de leitura e não o acúmulo do resultado, que acaba ficando na cache. 

A terceira versão (`multiply1_2`) usa a técnica de blocagem e com ela consegue melhorar bastante o desempenho, ficando limitada à vazão da cache L2 ou pouco mais. Note que a intensidade operacional volta a ser a inicial. Inicialmente, são usados blocos 16x16, experimente alterar este valor na constante [MATRIXTILESIZE](src/multiply.hpp) (precisa ser um divisor exato de 1024). O valor ideal vai depender da quantidade de cache do processador. 

## Referência

https://www.intel.com/content/www/us/en/develop/documentation/get-started-with-advisor/

Navegue pelas outras sessões, experimente com outras aplicações. 